# Multivariate linear

Multivariate linear regression is a statistical method used for modeling the relationship between multiple independent variables and a single dependent variable.
In contrast to [univariate linear regression](../univariate-linear), which involves only one independent variable, multivariate linear regression considers several simultaneously.
The goal is to create a linear equation that best fits the observed data, allowing for predictions or explanations of the dependent variable based on the values of the independent variables.

As always, let's load our CSV file.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
CSV_PATH = "https://gitlab.com/oasci/courses/pitt/biosc1540-2024s/-/raw/main/biosc1540/files/csv/advertising-data.csv"

df = pd.read_csv(CSV_PATH)

In the case of multivariate regression, we collect all of our independent variables in one dataframe called `df_features` and our dependent variable in `df_targets`.

In [3]:
target_column = "Product_Sold"

df_features = df.drop(columns=[target_column], inplace=False)
df_targets = df[target_column]

Now we need to convert the dataframe to NumPy arrays and reshape the targets.

In [4]:
targets = df_targets.to_numpy().reshape(-1, 1)
features = df_features.to_numpy()

## Linear

You actually use the same procedure to do multivariate regression.

In [5]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X=features, y=targets)
print(f"Intercept: {model.intercept_[0]:.3f}")
print(f"Coefficients: {model.coef_[0]}")

Intercept: 36.655
Coefficients: [1.97147823 2.79786525 1.59446751 2.43283307 1.40693022 3.91183385]


The intercept (`reg.intercept_`) is the constant term in the linear equation.
The intercept represents the estimated product sales when all advertising spending is zero.

The coefficients (`reg.coef_`) represent the weights assigned to each advertising channel (TV, Billboards, Google Ads, Social Media, Influencer Marketing, Affiliate Marketing) in predicting product sales.
These coefficients indicate the estimated change in product sales for a one-unit increase in each respective advertising channel, holding other variables constant.

Awesome!
We already performed a linear fit for `Social_Media` and found the coefficient to be `2.418`.
Let's check to see what it is now.

In [6]:
social_media_index = df_features.columns.get_loc("Social_Media")
print(model.coef_[0][social_media_index])

2.432833068526262


Wait a second, that is slightly different than before.
Which one is more accurate?

Well, let's see what the score is of this new model and compare it against our univariate score of `0.154`.

In [7]:
print(model.score(X=features, y=targets))

0.9401750192922066


Wow!
That is much better.




In helping the company, you can use these coefficients to guide them on how each advertising channel contributes to product sales.
For example, higher coefficients suggest a stronger positive impact on sales.
Adjusting ad budgets based on these coefficients could optimize their advertising strategy for higher sales.
Remember that correlation does not imply causation, so further analysis and experimentation may be needed to validate the findings.

## Normalization

In all of our analyses, we have not been consistently normalizing our data before fitting regression models.
This is a huge (intentional) oversight, and this raises concerns about the potential bias and misinterpretation of variable contributions in our models.
Without normalization, features with different scales may have been influencing our regression coefficients disproportionately, leading to skewed results.
Recognizing the importance of feature scale consistency in regression analysis, it becomes imperative for us to revisit our modeling approach and incorporate proper normalization techniques.
By doing so, we aim to enhance the accuracy and reliability of our models, ensuring that the impact of each predictor is appropriately considered, regardless of its original scale.

In [8]:
print(f"Mean: {np.mean(features, axis=0)}")
print(f"Stdev: {np.std(features, axis=0)}")

Mean: [515.9102   503.7412   518.42545  500.7092   483.258675 499.808   ]
Stdev: [274.58593496 260.36497289 274.35957055 266.42971192 277.79795289
 270.41370242]


We see that our data is more or less on the same scale, so we should be fine.
However, just to play it safe, let's normalize our features using [`MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) which scales and translates each feature individually such that it is in the given range on the training set, e.g. between zero and one.

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
features_normed = scaler.fit_transform(features)

print(f"Mean: {np.mean(features_normed, axis=0)}")
print(f"Stdev: {np.std(features_normed, axis=0)}")

Mean: [0.51638838 0.50430195 0.51156115 0.49673347 0.48294264 0.50269973]
Stdev: [0.2753956  0.26254674 0.2787159  0.27063264 0.27805933 0.27569604]


sklearn provides another scaler, called [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) that removes the mean and scaling to unit variance.

How do you choose which one to use?
In practice, it's often a good idea to try both scalers and observe the impact on your model's performance through cross-validation.
The choice between the two depends on the characteristics of your data and the assumptions of the models you are using.

**`StandardScaler`**

-   **When the distribution of the features is approximately Gaussian (normal):**
    `StandardScaler` assumes that your data follows a normal distribution. If your features are roughly normally distributed, using `StandardScaler` is a good choice.
-   **When you are using models that rely on distances between data points:**
    Models like k-nearest neighbors (KNN) and support vector machines (SVM) that involve distance calculations may perform better with standardized features.
-   **When dealing with algorithms that assume zero-mean and unit-variance features:**
    Some algorithms, like principal component analysis (PCA), assume that the features have zero mean and unit variance.
    Standardizing the features using `StandardScaler` ensures that these assumptions are met.

**`MinMaxScaler`**

-   **When your features have varying scales and you want to preserve the original distribution:**
    Use `MinMaxScaler` when you have features with different scales, and you want to scale them to a specific range (e.g., [0, 1]) while preserving the original distribution.
-   **For algorithms sensitive to the scale of input features:**
    Some algorithms, like neural networks and gradient-based optimization methods, can benefit from input features that are on a similar scale.
    In such cases, using `MinMaxScaler` might be a good choice.
-   **When you want to interpret the scaled features in the original domain:**
    If you need to interpret the scaled features in the context of the original data range, `MinMaxScaler` is more suitable.

## Polynomial


[`PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) will convert our individual features into all possible polynomial combinations.
For example, if we had two features $a$ and $b$, `PolynomialFeatures` would return the features $a$, $b$, $a^2$, $ab$, and $b^2$.

Let's proceed with our example data.

In [10]:
from sklearn.preprocessing import PolynomialFeatures

poly_degree = 2

# poly_featurizer is a class that will convert features
poly_featurizer = PolynomialFeatures(degree=poly_degree)

features_poly = poly_featurizer.fit_transform(features_normed)

print(f"n features: {features_normed.shape[1]}")
print(f"n poly features: {features_poly.shape[1]}")

n features: 6
n poly features: 28


Okay, we went from 6 to 28 features.
Curious as to what they are?

In [11]:
feature_names = poly_featurizer.get_feature_names_out()
print(feature_names)

['1' 'x0' 'x1' 'x2' 'x3' 'x4' 'x5' 'x0^2' 'x0 x1' 'x0 x2' 'x0 x3' 'x0 x4'
 'x0 x5' 'x1^2' 'x1 x2' 'x1 x3' 'x1 x4' 'x1 x5' 'x2^2' 'x2 x3' 'x2 x4'
 'x2 x5' 'x3^2' 'x3 x4' 'x3 x5' 'x4^2' 'x4 x5' 'x5^2']


1. Original features: $x_1, \; x_2, \; x_3, \; x_4, \; x_5, \; x_6$
2. Squared features: $x_1^2, \; x_2^2, \; x_3^2, \; x_4^2, \; x_5^2, \; x_6^2$
3. Product features: $x_1x_2, \; x_1x_3, \; x_1x_4, \; x_1x_5, \; x_1x_6, \; x_2x_3, \; x_2x_4, \; x_2x_5, \; x_2x_6, \; x_3x_4, \; x_3x_5, \; x_3x_6, \; x_4x_5, \; x_4x_6, \; x_5x_6$

We have already lost a lot of interpretability, as it is difficult to discern each feature's individual contributions.
Let's proceed anyway.

What is that `1` in there?
Well, that is the bias and this allows you to have a non-zero intercept.
This means we do not want our `LinearRegression` to fit another intercept, so we have to use `fit_intercept=False`.

In [12]:
model_poly = LinearRegression(fit_intercept=False)

# Fit the model with the polynomial features
model_poly.fit(features_poly, targets)

LinearRegression(fit_intercept=False)

In [13]:
for feature_name, coeff in zip(feature_names, model_poly.coef_[0]):
    print(f"{feature_name}: {coeff:.3e}")

1: 1.135e+02
x0: 2.159e+03
x1: 3.406e+03
x2: 1.150e+03
x3: 2.232e+03
x4: 1.096e+03
x5: 3.952e+03
x0^2: -3.152e+02
x0 x1: -3.633e+02
x0 x2: 2.983e+02
x0 x3: -6.785e+01
x0 x4: 4.945e+01
x0 x5: 3.729e+02
x1^2: 1.948e+02
x1 x2: -4.661e+02
x1 x3: -3.036e+02
x1 x4: -1.073e+02
x1 x5: -3.897e+02
x2^2: 2.727e+02
x2 x3: 2.408e+02
x2 x4: 2.480e+02
x2 x5: -2.628e+01
x3^2: 7.360e+01
x3 x4: 2.979e+02
x3 x5: -1.230e+01
x4^2: -4.403e+01
x4 x5: 1.977e+02
x5^2: -1.815e+02


Let's compare our model score to the standard linear regression of `0.94018`

In [14]:
score_poly = model_poly.score(X=features_poly, y=targets)
print(f"Poly score: {score_poly:.5f}")

Poly score: 0.94243


We see that our polynomial model has a slightly better $R^2$ score, but its not substantial.
However, now we do not have an easily interpretable model to recommend our business.

## Ridge

Ridge regression, also known as Tikhonov regularization or $L2$ regularization, is a linear regression technique that introduces a regularization term to the linear regression objective function.
The primary goal of ridge regression is to address the issue of multicollinearity in linear regression models.

### Objective function


In ridge regression, a regularization term is added to this objective function.
The new objective function becomes:

$$
\text{minimize} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 + \alpha \sum_{j=1}^{p} \beta_j^2
$$

Here:

-   $n$ is the number of observations.
-   $p$ is the number of predictors (features).
-   $y_i$​ is the actual value of the dependent variable for the ii-th observation.
-   $\hat{y}_i$​ is the predicted value of the dependent variable for the ii-th observation.
-   $\alpha$ is the regularization parameter, controlling the strength of regularization.
-   $\beta_j$ represents the coefficients of the linear regression model.


### Limitations


**Not Suitable for Feature Selection**

One of the primary limitations of ridge regression lies in its inability to perform feature selection.
Unlike some other regularization techniques, ridge regression retains all features in the model, making it less suitable for scenarios where variable selection is a critical requirement.

**Interpretability Challenges**

While ridge regression addresses multicollinearity, it does so at the expense of straightforward interpretability.
The regularization term introduces a level of complexity to coefficient interpretation, as the coefficients are shrunk towards zero.
This departure from the clear interpretability of standard linear regression should be acknowledged.

**Dependency on Scaling**

The performance of ridge regression is influenced by the scale of the variables.
Rescaling or standardizing the features is often necessary to ensure effective regularization.
Failure to do so can result in unequal penalization of coefficients, impacting the model's stability.

**No Sparsity in Coefficients**

Unlike some regularization methods, such as LASSO (L1 regularization), ridge regression does not lead to exact zero coefficients.
It shrinks coefficients towards zero but retains all features in the model.
If sparsity is a critical consideration, alternative regularization methods may be more appropriate.

**Model Complexity**

Ridge regression introduces a level of model complexity, with the choice of the regularization parameter ($\alpha$) playing a pivotal role.
Selecting an optimal $\alpha$ value often involves techniques like cross-validation, adding an additional layer of complexity to model tuning.

**Assumption of Linearity**

Similar to standard linear regression, ridge regression assumes a linear relationship between the independent and dependent variables.
If the true relationship is significantly nonlinear, ridge regression may not capture the underlying patterns accurately.

**Limited Handling of Multicollinearity**

While ridge regression is effective in mitigating multicollinearity, it may not completely eliminate the issue.
In cases of severe multicollinearity, additional techniques or data preprocessing methods may be necessary.

**Sensitivity to Outliers**

Ridge regression exhibits reduced sensitivity to outliers compared to standard linear regression, yet extreme values can still influence the regularization process and impact the resulting coefficients.
Practitioners should exercise caution in the presence of outliers.

**Not a Panacea**

It is crucial to recognize that ridge regression is not a universal solution.
Its effectiveness depends on the specific characteristics of the data and the objectives of the analysis.
Careful consideration of alternative regularization techniques may be warranted in certain scenarios.

In [15]:
from sklearn.linear_model import Ridge

model_ridge = Ridge(alpha=1.0)
model_ridge.fit(X=features_normed, y=targets)

print(f"Intercept: {model_ridge.intercept_[0]:.3f}")
print(f"Coefficients: {model_ridge.coef_[0]}")

Intercept: 372.860
Coefficients: [1906.99723949 2674.42209007 1513.78849822 2315.60502501 1348.95403981
 3701.14415754]


In [16]:
print(model_ridge.score(X=features_normed, y=targets))

0.9390368174983559
